In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['uy.pickle', 'uX.pickle']


In [4]:
import numpy as np
import re
import pickle 
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Unpickling dataset
X_in = open('../input/uX.pickle','rb')
y_in = open('../input/uy.pickle','rb')
X = pickle.load(X_in)
y = pickle.load(y_in)

In [6]:
type(X), len(X)

(list, 2000)

In [7]:
X[10]

b'it\'s ironic that the best films in cinema history are invariably the original director\'s cut of the film . \nfilms such as aliens , the abyss , the wild bunch , blade runner , and terminator 2 are all prime examples of a filmmaker\'s integrity , later chopped up or mucked with by the studio . \nthe advent of the dvd format has provided a more accessible way to get these original cuts to the public and provide to film freaks like myself the ability to become further enraptured by the extension of such classic films . \nthe dvd release of the original international version of luc besson\'s 1995 masterpiece the professional , which is known as l ? on around the world , is a prime example of how a good film can become an instant classic as a director\'s cut . \nfor years , i have heard of an " international " version available only in laserdisc format , which has eluded me for years . \ni even bought a laserdisc player from my uncle don for 100 bucks just to watch certain directors\' c

In [8]:
y

array([0, 1, 1, ..., 1, 0, 0])

In [9]:
# Creating the corpus
corpus = []
for i in range(0, 2000):
    review = re.sub(r'\W', ' ', str(X[i]))
    review = review.lower()
    review = re.sub(r'^br$', ' ', review)
    review = re.sub(r'\s+br\s+',' ',review)
    review = re.sub(r'\s+[a-z]\s+', ' ',review)
    review = re.sub(r'^b\s+', '', review)
    review = re.sub(r'\s+', ' ', review)
    corpus.append(review)  

In [10]:
corpus[5]

'don know how many other people have had the idea cross their mind that their life could be an ongoing television show watched by another world of people but it something used to wonder about when was younger ni can decide if first thought it because watched lot of tv or because my brother hit me in the head with baseball bat but m pretty sure andrew niccol screenwriter for the truman show has had the same curious thought nthe truman show is about man jim carrey whose entire life has been engineered by corporation and marketed to the public nsince birth he been living in the fictional island town of seahaven fla which actually exists as giant domed set just beyond the hollywood sign nall the people in truman burbank life are actors and the anonymous townfolk paid extras npeople watch truman life 24 hours day live with no commercial interruptions nrevenue comes instead from product placement staple of contemporary hollywood with truman friends and relatives describing their consumer ite

In [11]:
##Bag of Words
# Creating the BOW model
# min_df = 3      a word should pass at least 3 doc
# max_df = 0.6    words that pass in more than %60 of words is also eliminated

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 2000, min_df = 3, max_df = 0.6, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(corpus).toarray()

In [12]:
X.shape

(2000, 2000)

In [13]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [2, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [14]:
# Creating the Tf-Idf model directly
##The Math
##TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
##IDF(t) = log_e(Total number of documents / Number of documents with term t in it).
##Value = TF * IDF
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 2000, min_df = 3, max_df = 0.6, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(corpus).toarray()

In [15]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
text_train, text_test, sent_train, sent_test = train_test_split(X, y, test_size = 0.20, random_state = 0)


In [16]:
text_train.shape

(1600, 2000)

In [17]:
# Training the classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(text_train,sent_train)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [18]:
# Testing model performance
sent_pred = classifier.predict(text_test)


from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(sent_test, sent_pred)
cm

array([[168,  40],
       [ 21, 171]])

In [19]:
accuracy_score(sent_test, sent_pred)

0.8475

In [21]:
# Saving our classifier
with open('myclassifier.pickle','wb') as f:
    pickle.dump(classifier,f)
    
# Saving the Tf-Idf model
with open('mytfidfmodel.pickle','wb') as f:
    pickle.dump(vectorizer,f)

In [23]:
# Using our classifier
with open('mytfidfmodel.pickle','rb') as f:
    tfidf = pickle.load(f)
    
with open('myclassifier.pickle','rb') as f:
    clf = pickle.load(f)
    

In [24]:
sample = ["You are a bad person man, go to hell"]
sample = tfidf.transform(sample).toarray()
sentiment = clf.predict(sample)
sentiment

array([0])

In [25]:
sample = ["You are a nice person man, have a good life"]
sample = tfidf.transform(sample).toarray()
sentiment = clf.predict(sample)
sentiment

array([1])